# Read in fund performance spreadsheet from web, extract top 5 funds in each category & export to excel, extract protfolio fund prices, calculate  balance and append row to IBM Db2 database on a daily basis

In [1]:
import pandas as pd
import os
import ibm_db
from datetime import datetime as dt

In [2]:
# Set today's date
datenow = int(dt.today().strftime('%Y%m%d'))

In [3]:
# Configure DSN String for DataBase Connection

dsn_hostname = "dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net"
dsn_uid = ""
dsn_pwd = ""
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_port = "50000"
dsn_protocol = "TCPIP"

dsn = (
"DRIVER={0};"
"DATABASE={1};"
"HOSTNAME={2};"
"PORT={3};"
"PROTOCOL={4};"
"UID={5};"
"PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

In [4]:
# Define dictionary of funds and amount of holdings
assets = {85112:1, 96706:1, 97360:1, 97685:1, 98034:1, 98172:1, 98637:1}

## Define source of excel & read table into pandas dataframe

In [5]:
url='http://lt.morningstar.com/api/rest.svc/security_list/n85rs8kzci/FOALL$$ALL_1509?pageSize=5000&outputType=xlsx&languageId=en-GB&columnList=SecId,ISIN,CustomInstitutionSecurityId,Name,CustomCategoryName,ClosePrice,Offer,_CurrencyId,ReturnD1,ReturnW1,ReturnM1,ReturnM3,ReturnM6,ReturnM12,ReturnM36UA,ReturnM60UA,ActualManagementFee,ExpenseRatio&cacheBuster=4357814118873875000'

In [6]:
# Read date into dataframe, drop unwanted column, set index & sort by new index
table = pd.read_excel(url, index_col='Custom Institution Security Id')
table.drop('Total', axis=1, inplace=True)
table.sort_index(inplace=True)

## Explore Table Information

In [ ]:
table.shape

In [ ]:
table.info()

In [ ]:
table.describe()

In [ ]:
table.head()

It is clear from the breakdown below that there are only  7 main categories of funds: 
Equities, Fixed Income, Commodities, Multi-Asset,  Cash, Alternatives & Property

In [ ]:
table['Custom Category Name'].value_counts()
#table['Custom Category Name'].unique()

## Report the top 5 funds in each category sorted by 6 Month Return


In [7]:
# EQUITIES
equities = table[(table['Custom Category Name'].str.startswith("Equities")) & (table.Currency=="CU$$$$$USD")]
equities = equities.sort_values(by='6 Month Return', ascending=False).head()
#equities

In [8]:
# FIXED INCOME
fixed_income = table[(table['Custom Category Name'].str.startswith("Fixed Income")) & (table.Currency=="CU$$$$$USD")]
fixed_income = fixed_income.sort_values(by='6 Month Return', ascending=False).head()
#fixed_income

In [9]:
# COMMODITIES
commodities = table[(table['Custom Category Name'].str.startswith("Commodities")) & (table.Currency=="CU$$$$$USD")]
commodities = commodities.sort_values(by='6 Month Return', ascending=False).head()
#commodities

In [10]:
# MULTI-ASSET
multi_asset = table[(table['Custom Category Name'].str.startswith("Multi-Asset")) & (table.Currency=="CU$$$$$USD")]
multi_asset = multi_asset.sort_values(by='6 Month Return', ascending=False).head()
#multi_asset

In [11]:
# CASH FUNDS
cash_funds = table[(table['Custom Category Name'].str.startswith("Cash Funds")) & (table.Currency=="CU$$$$$USD")]
cash_funds = cash_funds.sort_values(by='6 Month Return', ascending=False).head()
#cash_funds

In [12]:
# ALTERNATIVES
alternatives = table[(table['Custom Category Name'].str.startswith("Alternatives")) & (table.Currency=="CU$$$$$USD")]
alternatives = alternatives.sort_values(by='6 Month Return', ascending=False).head()
#alternatives

In [13]:
# PROPERTY
realestate = table[(table['Custom Category Name'].str.startswith("Property")) & (table.Currency=="CU$$$$$USD")]
realestate = realestate.sort_values(by='6 Month Return', ascending=False).head()
#realestate

## Show Funds with Maximum Returns over 6 reporting periods: 1M, 3M, 6M, 1Y, 3Y, 5Y

In [14]:
# Define variables to locate rows with maximum returns
ID1M, ID3M, ID6M, ID12M, ID3Y, ID5Y = table['Return 1M'].idxmax(), table['Return 3M'].idxmax(), table['6 Month Return'].idxmax(), table['Return 12M'].idxmax(), table['Return 36M UA'].idxmax(), table['5 Year Return (unannualised)'].idxmax()

In [15]:
# Optional Visual Check of Funds with Maximum Performance
print('Max 1M Return: {} - {}, with a value of %{}'.format(ID1M, table.loc[ID1M,'Name'], table.loc[ID1M,'Return 1M']))
print('Max 3M Return: {} - {}, with a value of %{}'.format(ID3M, table.loc[ID3M,'Name'], table.loc[ID3M,'Return 3M']))
print('Max 6M Return: {} - {}, with a value of %{}'.format(ID6M, table.loc[ID6M,'Name'], table.loc[ID6M,'6 Month Return']))
print('Max 1Y Return: {} - {}, with a value of %{}'.format(ID12M, table.loc[ID12M,'Name'], table.loc[ID12M,'Return 12M']))
print('Max 3Y Return: {} - {}, with a value of %{}'.format(ID3Y, table.loc[ID3Y,'Name'], table.loc[ID3Y,'Return 36M UA']))
print('Max 5Y Return: {} - {}, with a value of %{}'.format(ID5Y, table.loc[ID5Y,'Name'], table.loc[ID5Y,'5 Year Return (unannualised)']))

Max 1M Return: 98582 - QIN IM USD Merian Gold & Silver, with a value of %24.92308
Max 3M Return: 97285 - QIN IM EUR BNP Paribas Energy Transition USD, with a value of %82.51534
Max 6M Return: 97464 - QIN IM USD Investec GSF Global Gold, with a value of %55.46995
Max 1Y Return: 97285 - QIN IM EUR BNP Paribas Energy Transition USD, with a value of %72.96512
Max 3Y Return: 97464 - QIN IM USD Investec GSF Global Gold, with a value of %103.018125
Max 5Y Return: 97465 - QIN IM USD Investec GSF Global Gold GBP, with a value of %279.024374


In [16]:
top6indexlist=[ID1M, ID3M, ID6M, ID12M, ID3Y, ID5Y]
top_performers = table[table.index.isin(top6indexlist)]
top_performers = top_performers.drop(['Return 1D', 'Return 1W'], axis=1)
#top_performers

## Export Top Performing Funds to multi-sheet excel workbook

In [17]:
filename = str(datenow)+'_Top Funds.xlsx'
full_filename = os.path.join('/Users/Richard/Dropbox/RSK/', filename)

with pd.ExcelWriter(full_filename) as writer:
    top_performers.to_excel(writer, sheet_name="Top Funds")
    cash_funds.to_excel(writer, sheet_name="Cash")
    commodities.to_excel(writer, sheet_name="Commodities")
    equities.to_excel(writer, sheet_name="Equities")
    fixed_income.to_excel(writer, sheet_name="Fixed Income")
    multi_asset.to_excel(writer, sheet_name="MultiAsset")
    alternatives.to_excel(writer, sheet_name="Alternatives")
    realestate.to_excel(writer, sheet_name="Property")  

## Generate new DataFrame for funds in portfolio ONLY

In [18]:
portfolio = table[table.index.isin(assets.keys())]

In [ ]:
# Optional Visual check of Portfolio DataFrame
#portfolio

## Add total value for each Fund Row

In [19]:
# Create empty list, loop through calculation, append values to DataFrame
value_list=[]
for index in portfolio.index:
    value_list.append(round(portfolio.loc[index,'Close Price'] * assets[index],2))
portfolio.insert(17, 'Value', value_list)

## Compare max date from Database to today's date and delete row if data already exists

In [34]:
conn = ibm_db.connect(dsn, "", '')
readQuery = "select max(DATE) from qin_portfolio"
readStmt = ibm_db.exec_immediate(conn, readQuery)
last_db_date = ibm_db.fetch_both(readStmt)[0]

if datenow == last_db_date:
    delQuery = "delete from qin_portfolio where DATE = "+str(datenow)
    delStmt = ibm_db.exec_immediate(conn, delQuery)

## Update Database with daily closing prices and total portfolio value

In [31]:
# Define variables for SQL Update Statement
#
QIN_85112 = portfolio.loc[85112, 'Close Price']
QIN_96706 = portfolio.loc[96706, 'Close Price']
QIN_97360 = portfolio.loc[97360, 'Close Price']
QIN_97685 = portfolio.loc[97685, 'Close Price']
QIN_98034 = portfolio.loc[98034, 'Close Price']
QIN_98172 = portfolio.loc[98172, 'Close Price']
QIN_98637 = portfolio.loc[98637, 'Close Price']
dailysum = round(sum(portfolio.Value),2)

## Connecting to IBM Db2 Database via IBM_DB API in Python - OPTION 1

In [35]:
insertQuery ='''
insert into qin_portfolio
(date, "85112", "96706", "97360", "97685", "98034", "98172", "98637", daysum)
VALUES
('''+str(datenow)+","+str(QIN_85112)+","+str(QIN_96706)+","+str(QIN_97360)+","+str(QIN_97685)+","+str(QIN_98034)+","+str(QIN_98172)+","+str(QIN_98637)+","+str(dailysum)+")"

insertStmt = ibm_db.exec_immediate(conn, insertQuery)
ibm_db.close(conn)

In [ ]:
# Optional: Check rows in Online DB
conn = ibm_db.connect(dsn, "", '')
selectQuery = "select * from qin_portfolio"
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

In [ ]:
# Optional: Print summary of database
while ibm_db.fetch_row(selectStmt) != False:
    print('Date: ', ibm_db.result(selectStmt, 'DATE'), 'Value: ', ibm_db.result(selectStmt, 'DAYSUM'))
ibm_db.close(conn)

## Connecting to IBM Db2 Database via SQL Magic - OPTION 2

In [ ]:
%load_ext sql

In [ ]:
%sql ibm_db_sa://USR:PSWD@dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net:50000/BLUDB

In [ ]:
# CODE TO CREATE TABLE IN SQL
# ONLY RUN ONCE
#%%sql
#create table QIN_PORTFOLIO
#(date INTEGER PRIMARY KEY NOT NULL,
# "85112" DECIMAL (10,2),
# "96706" DECIMAL (10,2),
# "97360" DECIMAL (10,2),
# "97685" DECIMAL (10,2),
# "98034" DECIMAL (10,2),
# "98172" DECIMAL (10,2),
# "98637" DECIMAL (10,2),
# daysum DECIMAL (10,2)
#)

In [ ]:
%%sql
insert into qin_portfolio
(date, "85112", "96706", "97360", "97685", "98034", "98172", "98637", daysum) 
VALUES
(:datenow, :QIN_85112, :QIN_96706, :QIN_97360, :QIN_97685, :QIN_98034, :QIN_98172, :QIN_98637, :dailysum)


## Optional : Verification of Addition to Database

In [26]:
#%load_ext sql

In [27]:
#%sql ibm_db_sa://USR:PSWD@dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net:50000/BLUDB

In [38]:
#%sql select * from qin_portfolio

In [39]:
#%sql select tabname, tabschema, create_time from syscat.tables where tabschema='RRB73189'

In [40]:
#%sql select * from syscat.columns where tabname='QIN_PORTFOLIO'

In [41]:
#%sql delete from qin_portfolio where date=20200810